# Workflow Integration

> End-to-end workflow: Finnish texts → Translation → Audio → Anki cards

In [ ]:
#| default_exp workflow

In [ ]:
#| export
from pathlib import Path

from suomi.core import *
from suomi.tsv import *
from suomi.mp3 import *
from suomi.anki import *

## Main Workflow Function

In [ ]:
#| export
def create_cards(
    texts: list[str],
    deck: str,
    tags: str | list[str] = "lang::fi",
    output_dir: str = "output",
    overwrite: bool = False,
) -> Path:
    """
    Create Anki cards from Finnish texts (end-to-end workflow).
    
    Steps:
    1. Translate Finnish → English/Japanese (OpenAI API)
    2. Generate audio files (Piper TTS)
    3. Update TSV with file paths
    4. Upload to Anki (AnkiConnect)
    
    Args:
        texts: List of Finnish words/phrases to create cards for
        deck: Anki deck name (e.g., "06::Daily")
        tags: Tags for the cards (string or list), hierarchical format supported
              Examples: "src::daily", "src::class,level::A1", ["src::medical", "urgent"]
              Note: "lang::fi" is always auto-included
        output_dir: Directory for TSV and audio files (default: "output")
        overwrite: If True, delete existing deck and overwrite TSV file
    
    Example:
        >>> create_cards(
        ...     texts=["kissa", "koira"],
        ...     deck="06::Daily",
        ...     tags="src::daily"
        ... )
    """
    
    tsv_path = f"{output_dir}/{Path(deck.replace("::", "_")).stem + ".tsv"}"
    if overwrite:
        if deck in deckNames():
            deleteDeck(deck)
        # TSV will be auto-overwritten by texts2tsv()
    else:
        if Path(tsv_path).exists():
            raise FileExistsError(
                f"TSV file already exists: {tsv_path}\n"
                f"Use overwrite=True to replace it."
            )
        if deck in deckNames():
            raise ValueError(
                f"Deck '{deck}' already exists in Anki.\n"
                f"Use overwrite=True to replace it."
            )
        
    audio_dir = f"{output_dir}/audio"
    images_dir = f"{output_dir}/images"
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    texts2tsv(texts, tsv_path, tags=tags)
    mp3s(tsv_path, output_dir=audio_dir)
    update_tsv_media_paths(tsv_path, dirs=[audio_dir, images_dir])
    addnotes(deck, tsv_path)
    return tsv_path

## Example Usage

In [ ]:
#| eval: false
examples = [
    # {
    #     "path": "texts/Ahaa!::05::Lääkarissä.txt",
    #     "tags": "lang::fi::Lääkarissä,Ahaa!::05::Lääkarissä"
    # },
    # {
    #     "path": "texts/Ahaa!::05::Kasvot.txt", 
    #     "tags": "lang::fi::Kasvot,Ahaa!::05::Lääkarissä"
    # },
    # {
    #     "path": "texts/Ahaa!::05::Oireet.txt", 
    #     "tags": "lang::fi::Oireet,Ahaa!::05::Lääkarissä"
    # },
    # {
    #     "path": "texts/Vartalo::Jooga.txt", 
    #     "tags": "lang::fi::Vartalo,Jooga"
    # },
    # {
    #     "path": "texts/Kuukaudet_Vuodenajat.txt", 
    #     "tags": "lang::fi::Kuukaudet,lang::fi::Vuodenajat"
    # },
    # {
    #     "path": "texts/Järjestysluvut.txt", 
    #     "tags": "lang::fi::Järjestysluvut"
    # },
    {
        "path": "texts/Yksikkö_Monikko.txt", 
        "tags": "lang::fi::Monikko"
    },
    # {
    #     "path": "texts/Cafe.txt",
    #     "tags": "lang::fi::cafe"
    # },
    # {
    #     "path": "texts/Avanto.txt",
    #     "tags": "lang::fi::avanto"
    # },
]

for ex in examples:
    txt = Path(ex["path"])
    tsv_path = create_cards(
        texts=cattxt(txt),
        deck=txt.stem,
        tags=ex["tags"],
        overwrite=True,
    )    
    print(f"{'='*60}\n📝 TXT: {txt} → TSV: {tsv_path}")
    rows, _, _ = cattsv(tsv_path)
    for idx, row in enumerate(rows[-2:]):
        print(f"  [{idx}] {row['Finnish'][:20]} → {row['English'][:20]} Audio: {row['mp3_path']}")

📝 TXT: texts/Yksikkö_Monikko.txt → TSV: output/Yksikkö_Monikko.tsv
  [0] Vastaus on vihkossa. → The answer is in the Audio: output/audio/Yksikkö_Monikko_35.mp3
  [1] Vastaukset ovat vihk → The answers are in t Audio: output/audio/Yksikkö_Monikko_36.mp3
